### 模型int8量化及轉換成tflite檔。
- used in alarm, moaning, help and misc-sound classification.

### QAT過程問題集
- can't convert float NaN (actually 0.00000) to int:
  - 與weight-decay設定值可能有關，設定太大倒導致錯誤。
  - 可能是同時開啟三個訓練程式造成記憶體不足造成。
- 訓練時必需要用cpu不能用cuda，原因是qnnpack不支援。
- 若出現labels values error，可能是因為載入的batch size超過記憶體，將batch size降低即可。

In [26]:
import sys;
import os;
import glob;
import math;
import random;
import torch;
import torch.optim as optim;
import torch.nn as nn;
import numpy as np
import copy

In [27]:
print(torch.__version__)

2.4.0+cu118


In [28]:
project_PATH = '/home/sail/sound_project/sound_ai_v2'
step_now = 'STEP/step_5'

use_model_in_step4 = "/home/sail/sound_project/sound_ai_v2/STEP/step_4/save_pt_model_s4/20240901_04/retrain_4C_after_2nd_pruning_prunratio0.9_2024090104/retrained_4Cmodel_ratio90.0_vaacc88.0_tracc_80.6640625@1287epoch_20240901071124.pt"


In [29]:
import os;
import sys;
sys.path.append(os.path.abspath(f'{project_PATH}'))



In [30]:
import common.opts as opts;
import common.utils as U;
# import th.resources.models as models;
import th.resources.no_softmax_quant_model_layer_cut as models;
# import th.resources.models_layer_cut as models;
import th.resources.calculator as calc;
from SharedLibs.datestring import getDateStr, genDataTimeStr;

In [31]:
import argparse
import time
from datetime import datetime;


In [32]:
from torch.quantization import QuantStub, DeQuantStub
import torch.nn.functional as F

from tinynn.converter import TFLiteConverter
from tinynn.graph.quantization.quantizer import PostQuantizer
from tinynn.graph.tracer import model_tracer
from tinynn.util.train_util import DLContext, get_device
from tinynn.graph.quantization.algorithm.cross_layer_equalization import cross_layer_equalize

In [33]:
inp_len = 20150
sr = 20150
choose_class=[0,1,2,3,4,5,6]
PATH = f'{project_PATH}/STEP/data_v2.npz'

ch_n_class = 7
fcn_no_of_inputs = 7

In [34]:
date_time = datetime.now().strftime("%Y%m%d_%H")  # %M
save_pt_model_path = f'{project_PATH}/{step_now}/save_pt_model_s5/{date_time}'

save_tflite_model_path = f'{project_PATH}/{step_now}/save_tflite_model_s5/{date_time}'
try:
    os.mkdir(save_pt_model_path)
    print(f"Folder '{save_pt_model_path}' created successfully.")
except FileExistsError:
    print(f"Folder '{save_pt_model_path}' already exists.")

try:
    os.mkdir(save_pt_model_path+'/uncompressed_qat_models')
    print(f"Folder '{save_pt_model_path}' created successfully.")
except FileExistsError:
    print(f"Folder '{save_pt_model_path}' already exists.")

    
try:
    os.mkdir(save_tflite_model_path)
    print(f"Folder '{save_tflite_model_path}' created successfully.")
except FileExistsError:
    print(f"Folder '{save_tflite_model_path}' already exists.")

Folder '/home/sail/sound_project/sound_ai_v2/STEP/step_5/save_pt_model_s5/20240902_01' already exists.
Folder '/home/sail/sound_project/sound_ai_v2/STEP/step_5/save_pt_model_s5/20240902_01' already exists.
Folder '/home/sail/sound_project/sound_ai_v2/STEP/step_5/save_tflite_model_s5/20240902_01' already exists.


In [35]:
data = np.load(PATH, allow_pickle=True) 
audio_max_value =  float((data['sounds_train']).max())*2 # the max value of the sound samples
audio_min_value =  float((data['sounds_train']).min())*2 # the max value of the sound samples
audio_max_value, audio_min_value

(3.0242209434509277, -3.1673121452331543)

In [36]:
seed = 1123;
random.seed(seed);
np.random.seed(seed);
torch.manual_seed(seed);
if torch.cuda.is_available():
    torch.cuda.manual_seed(seed);
torch.backends.cudnn.deterministic = True;
torch.backends.cudnn.benchmark = False;

In [37]:
order = [0,1,2,3,4,5,6]

def one_hot_encode(data, order):
    index_map = {number: index for index, number in enumerate(order)}
    one_hot_list = []

    for num in data:
        one_hot = [0] * len(order)
        if num in index_map:
            one_hot[index_map[num]] = 1
        one_hot_list.append(one_hot)

    return one_hot_list

In [38]:
mask8 = 0x4000 # >> 8 : 16384
mask7 = 0x2000 # >> 7 :  8192
mask6 = 0x1000 # >> 6 :  4096
mask5 = 0x0800 # >> 5 :  2048
mask4 = 0x0400 # >> 4 :  1024
mask3 = 0x0200 # >> 3 :   512
mask2 = 0x0100 # >> 2 :   256
mask1 = 0x0080 # >> 1 :   128
mask0 = 0x0040 # >> 0 :    64 below the value, drop the value

In [39]:
def maskOP(x):
    x = np.int16(x)
    # print(f"begin:x:{x}")
    if (mask8&x):
        return x >> 8
    elif (mask7&x):
        return x >> 7
    elif (mask6&x):
        return x >> 6
    elif (mask5&x):
        return x >> 5
    elif (mask4&x):
        return x >> 4
    elif (mask3&x):
        return x >> 3
    elif (mask2&x):
        return x >> 2
    elif (mask1&x):
        return x >> 1
    elif (mask0&x):
        return x
    else:
        return 0;

In [40]:
# def quantize_int8(x, axis):
#     len_of_x = len(x[0][0][0])
#     print(f"len_of_x:{len_of_x}")
#     for i in range(len_of_x):
#         nflag = 2; #positive
#         print("{}:{}".format(i,x[0][0][0][i]))
#         tmp_x = x[0][0][0][i]
#         if tmp_x < 0:
#             tmp_x = np.abs(tmp_x)
#             nflag = 1
#         tmp_x = maskOP(tmp_x)
#         if(nflag==1):
#             tmp_x = -1 * (tmp_x)
#         print("{}:{}".format(i,x[0][0][0][i]))
#         print("*********************************")
#         x[0][0][0][i] = tmp_x
#     return x

In [41]:
def quantize_int8(x):
    # Ensure x is a numpy array with float32 type
    x = np.array(x, dtype=np.float32)
    len_of_x = x.shape[3] # assuming x is at least 4D
    print(f"len_of_x: {len_of_x}")
    
    for i in range(len_of_x):
        nflag = 2  # positive
        print("{}:{}".format(i, x[0, 0, 0, i]))
        
        tmp_x = x[0, 0, 0, i]
        if tmp_x < 0:
            tmp_x = np.abs(tmp_x)
            nflag = 1
        
        tmp_x = maskOP(tmp_x)
        
        if nflag == 1:
            tmp_x = -1 * tmp_x
            
        print("{}:{}".format(i, tmp_x))
        print("*********************************")
        
        # Ensure the result is treated as a float before assignment
        x[0, 0, 0, i] = np.float32(tmp_x)
    
    return x

def maskOP(value):
    # Example implementation; replace with actual logic
    # This function should handle the logic of masking or adjusting the value
    # Ensure that this function is correctly implemented to output expected results
    return np.clip(value, -128, 127)  # Example clip for int8 range



In [42]:
def quantize_float32_2_int8(data, min_val=None, max_val=None):
    # Determine min and max values for scaling
    if min_val is None:
        min_val = data.min()
    if max_val is None:
        max_val = data.max()

    # Calculate the scale and zero-point
    scale = (max_val - min_val) / 255.0  # Scale down to the range that int8 can represent
    zero_point = 128  # Center around zero for int8 which is [-128, 127]

    # Apply quantization formula
    quantized_data = np.round((data - min_val) / scale).astype(np.int8) - zero_point

    # Clip to the valid int8 range
    quantized_data = np.clip(quantized_data, -128, 127)

    return quantized_data

In [43]:
class TLGenerator():
    #Generates data for Keras
    def __init__(self, samples, labels, options):
        random.seed(seed);
        #Initialization
        print(f"length of samples:{len(samples)}")
        self.data = [(samples[i], labels[i]) for i in range (0, len(samples))];
        self.opt = options;
        self.batch_size = options.batchSize;
        self.preprocess_funcs = self.preprocess_setup();
        self.mapdict ={0:1, 1:2, 2:3, 3:4, 4:5, 5:6, 6:7,}

    def __len__(self):
        #Denotes the number of batches per epoch
        return int(np.floor(len(self.data) / self.batch_size));

    def __getitem__(self, batchIndex):
        #Generate one batch of data
        batchX, batchY = self.generate_batch(batchIndex);
        batchX = np.expand_dims(batchX, axis=1);
        batchX = np.expand_dims(batchX, axis=3);
        return batchX, batchY

    def generate_batch(self, batchIndex):
        #Generates data containing batch_size samples
        sounds = [];
        labels = [];
        indexes = None;
        for i in range(self.batch_size):
            # Training phase of BC learning
            # Select two training examples
            while True:
                sound1, label1 = self.data[random.randint(0, len(self.data) - 1)]
                sound2, label2 = self.data[random.randint(0, len(self.data) - 1)]
                if label1 != label2:
                    break
            sound1 = self.preprocess(sound1)
            sound2 = self.preprocess(sound2)

            # Mix two examples
            r = np.array(random.random())
            sound = U.mix(sound1, sound2, r, self.opt.sr).astype(np.float32)
            # print(f"sound length after U.mix is {len(sound)}")
            eye = np.eye(self.opt.nClasses)
            idx1 = label1
            idx2 = label2
            label = (eye[idx1] * r + eye[idx2] * (1 - r)).astype(np.float32)
            # label = (eye[label1] * r + eye[label2] * (1 - r)).astype(np.float32)

            #For stronger augmentation
            sound = U.random_gain(6)(sound).astype(np.float32)
            # print(f"sound length after U.random_gain is {len(sound)}")
            sounds.append(sound);
            labels.append(label);

        sounds = np.asarray(sounds);
        labels = np.asarray(labels);
        # print(f"labels in generate_batch is:\n{labels}")

        return sounds, labels;

    def preprocess_setup(self):
        funcs = []
        if self.opt.strongAugment:
            funcs += [U.random_scale(1.25)]

        funcs += [U.padding(self.opt.inputLength // 2),
                  U.random_crop(self.opt.inputLength),
                  U.regularization(audio_max_value, audio_min_value)]
        return funcs

    def preprocess_setup_without_normalization(self):
        funcs = []
        if self.opt.strongAugment:
            funcs += [U.random_scale(1.25)]

        funcs += [U.padding(self.opt.inputLength // 2),
                  U.random_crop(self.opt.inputLength)
                  ]
        return funcs

    def preprocess(self, sound):
        for f in self.preprocess_funcs:
            sound = f(sound)

        return sound;

In [44]:
def getTrainGen(opt=None, split=None):
    
    dataset = np.load(opt.Data_npz_path, allow_pickle=True);
    train_sounds = []
    train_labels = []

    train_sounds = dataset['sounds_train']
    train_labels = dataset['labels_train']

    trainGen = TLGenerator(train_sounds, train_labels, opt);
    trainGen.preprocess_setup();
    return trainGen

In [45]:
def getOpts():
    parser = argparse.ArgumentParser(description='Transfer Learning for ACDNet');
    parser.add_argument('--netType', default='TLACDNet',  required=False);
    parser.add_argument('--data', default='./datasets/processed/',  required=False);
    parser.add_argument('--dataset', required=False, default='uec_iot', choices=['10']);
    parser.add_argument('--BC', default=True, action='store_true', help='BC learning');
    parser.add_argument('--strongAugment', default=True,  action='store_true', help='Add scale and gain augmentation');
    #在ipynb中，不能使用parser.parse，要改用parser.parse_known_args()
    opt, unknown = parser.parse_known_args()
    """
   
    """
    #Leqarning settings
    opt.batchSize = 32 # 48                                                                                                                                                                                                                                                            ;
    opt.weightDecay = 5e-4;
    opt.LR = 0.05;
    opt.momentum = 0.5;
    opt.nEpochs = 3000;
    opt.schedule = [0.1,0.3, 0.6, 0.9];
    opt.warmup = 10;
    opt.Data_npz_path = PATH

    #Basic Net Settings
    opt.nClasses = ch_n_class#50;
    opt.nFolds = 1;#5;
    opt.splits = [i for i in range(1, opt.nFolds + 1)];
    opt.sr = sr;
    opt.inputLength = inp_len;
    #Test data
    opt.nCrops = 2;
    # opt.ch_config = [8,64,32,64,64,128,128,256,256,512,512,2];
    return opt

In [46]:
def display_info(opt):
    print('+------------------------------+');
    print('| {} Sound classification'.format(opt.netType));
    print('+------------------------------+');
    print('| dataset  : {}'.format(opt.dataset));
    print('| nEpochs  : {}'.format(opt.nEpochs));
    print('| LRInit   : {}'.format(opt.LR));
    print('| schedule : {}'.format(opt.schedule));
    print('| warmup   : {}'.format(opt.warmup));
    print('| batchSize: {}'.format(opt.batchSize));
    print('| nFolds: {}'.format(opt.nFolds));
    print('| Splits: {}'.format(opt.splits));
    print('| Device: {}'.format(opt.device));
    print('| Model Path: {}'.format(opt.model_path));
    print('| Model Name: {}'.format(opt.model_name));
    print('+------------------------------+');

In [47]:
class QATTrainer:
    def __init__(self, opt=None, split=0):
        self.opt = opt;
        self.testX = None;
        self.testY = None;
        self.trainX = None;
        self.trainY = None;
        # self.opt.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu");
        self.opt.device = torch.device("cpu")
        self.trainGen = getTrainGen(self.opt)#train_generator.setup(self.opt, self.opt.split);
        self.qunt_nClass = opt.nClasses;
        self.bestAcc = 0.0;
        self.bestAccEpoch = 0;

    def load_train_data(self):
        print('Preparing calibration dataset..');
        # x,y = self.trainGen.__getitem__(0);
        # self.trainX = torch.tensor(np.moveaxis(x, 3, 1)).to(self.opt.device);
        # """
        # trainX size:torch.Size([1, 1, 30225]), but must be [1,1,1,30225]
        # Due to the reason: raise ValueError("Input shape must be `(N, C, H, W)`!")
        # """
        # # print(f"trainX[0] shape:{self.trainX[0].shape}")
        # self.trainY = torch.tensor(y).to(self.opt.device);
        data = np.load(self.opt.Data_npz_path, allow_pickle=True);
        print(f"device is :{self.opt.device}")
        print(f"len of Y:{len(data['labels_train'])}")
        dataX = data['sounds_train'].reshape(data['sounds_train'].shape[0],1,1,data['sounds_train'].shape[1]) # .astype(np.float32);
        self.trainX = torch.tensor(dataX).to(self.opt.device);
        self.trainY = torch.tensor(data['labels_train']).to(self.opt.device);          # .type(torch.float32)
        print('Calibration dataset is ready');


    def load_test_data(self):
        data = np.load(self.opt.Data_npz_path, allow_pickle=True);
        print(f"device is :{self.opt.device}")
        print(f"len of Y:{len(data['labels_val'])}")
        dataX = data['sounds_val'].reshape(data['sounds_val'].shape[0],1,1,data['sounds_val'].shape[1]).astype(np.float32);
        self.testX = torch.tensor(dataX).to(self.opt.device);
        self.testY = torch.tensor(one_hot_encode(data['labels_val'], order)).type(torch.float32).to(self.opt.device);

    def __validate_test(self, net, qat_done, testX, testY):
        net.eval();
        # if qat_done:
        #     testX.to('cpu');
        #     testY.to('cpu');
        # else:
        #     testX.to('cuda:0');
        #     testY.to('cuda:0');
            
        with torch.no_grad():
            y_pred = None;
            batch_size = len(self.testX);
            x = self.testX[:];
            scores = net(x);
            y_pred = scores.data if y_pred is None else torch.cat((y_pred, scores.data));
            acc = self.__compute_accuracy_2(y_pred, self.testY);
        return acc;

    
    def __validate(self, net, lossFunc):
        if self.testX is None:
            self.load_test_data();
        net.eval();
        acc=0.0; 
        loss = 0.0;
        with torch.no_grad():
            y_pred = None;
            batch_size = len(self.testX);#(self.opt.batchSize//self.opt.nCrops)*self.opt.nCrops;
            x = self.testX[:];
            try:
                scores = net(x);
                y_pred = scores.data if y_pred is None else torch.cat((y_pred, scores.data));
                acc, loss = self.__compute_accuracy(y_pred, self.testY, lossFunc);
            except ValueError:
                print(f"error data:val")
                # print(f"error data:{x}")
        net.train();
        return acc, loss;

    #Calculating average prediction (10 crops) and final accuracy
    def __compute_accuracy_2(self, y_pred, y_target):
        print(y_pred.shape);
        with torch.no_grad():
            y_pred = (y_pred.reshape(y_pred.shape[0]//self.opt.nCrops, self.opt.nCrops, y_pred.shape[1])).mean(dim=1);
            y_target = (y_target.reshape(y_target.shape[0]//self.opt.nCrops, self.opt.nCrops, y_target.shape[1])).mean(dim=1);

            y_pred = y_pred.argmax(dim=1);
            y_target = y_target.argmax(dim=1);

            acc = (((y_pred==y_target)*1).float().mean()*100).item();
        return acc;
        

    # def __compute_accuracy(self, y_pred, y_target, lossFunc):
    #     print(f"shape of y_pred:{y_pred.shape}");
    #     print(f"shape of y_target:{y_target.shape}");
        
    #     with torch.no_grad():
    #         #Reshape to shape theme like each sample comtains 10 samples, calculate mean and find theindices that has highest average value for each sample
    #         if self.opt.nCrops == 1:
    #             y_pred = y_pred.argmax(dim=1);
    #             y_target = y_target.argmax(dim=1);
    #         else:
    #             y_pred = (y_pred.reshape(y_pred.shape[0]//self.opt.nCrops, self.opt.nCrops, y_pred.shape[1])).mean(dim=1).argmax(dim=1);
    #             y_target = (y_target.reshape(y_target.shape[0]//self.opt.nCrops, self.opt.nCrops, y_target.shape[1])).mean(dim=1).argmax(dim=1);
    #             print(f"after: len of y_pred:{len(y_pred)}, len of y_target:{len(y_target)}")
    #         acc = (((y_pred==y_target)*1).float().mean()*100).item();
    #         # valLossFunc = torch.nn.KLDivLoss();
    #         loss = lossFunc(y_pred.float().log(), y_target.float()).item();
    #         # loss = 0.0;
    #     return acc, loss;
    def __compute_accuracy(self, y_pred, y_target, lossFunc):
        print("===========================")
        print(f"shape of y_pred: {y_pred.shape}")
        print(f"shape of y_target: {y_target.shape}")
        
        with torch.no_grad():
            if y_pred.ndim > 1:
                y_pred = F.log_softmax(y_pred, dim=-1)  # Apply log_softmax for KLDivLoss

            if y_pred.shape != y_target.shape:
                raise ValueError(f"Mismatched shapes y_pred: {y_pred.shape}, y_target: {y_target.shape}")

            # For accuracy, extract predicted classes
            y_pred_classes = y_pred.argmax(dim=1)
            acc = (((y_pred_classes == y_target.argmax(dim=1)) * 1).float().mean() * 100).item()

            # Ensure `y_target` is in a compatible shape or processing state
            loss = lossFunc(y_pred, y_target).item()  # Use y_pred already log_softmaxed

        return acc, loss
            

    def __load_model(self, quant=True):
        state = torch.load(self.opt.model_path, map_location=self.opt.device);
        print(state['config']);
        net = None;
        net = models.GetACDNetQuantModel(input_len=self.opt.inputLength, nclass=self.qunt_nClass, sr=self.opt.sr, channel_config=state['config']).to(self.opt.device);
        calc.summary(net, (1,1,self.opt.inputLength));
        net.load_state_dict(state['weight']);
        return net;

    
    def __train(self, net):
        self.load_train_data();
        # net.eval();
        # calc.summary(net, (1,1,self.opt.inputLength));
        lossFunc = torch.nn.KLDivLoss(reduction='batchmean');
        optimizer = optim.SGD(net.parameters(), lr=self.opt.LR, weight_decay=self.opt.weightDecay, momentum=self.opt.momentum, nesterov=True);
        train_start_time = time.time();
        for epochIdx in range(self.opt.nEpochs):
            epoch_start_time = time.time();
            optimizer.param_groups[0]['lr'] = self.__get_lr(epochIdx+1);
            cur_lr = optimizer.param_groups[0]['lr'];
            running_loss = 0.0;
            running_acc = 0.0;
            n_batches = math.ceil(len(self.trainGen.data)/self.opt.batchSize);
            for batchIdx in range(n_batches):
                # with torch.no_grad():
                x,y = self.trainGen.__getitem__(batchIdx)
                x = torch.tensor(np.moveaxis(x, 3, 1)).to(self.opt.device);
                y = torch.tensor(y).to(self.opt.device);
                # zero the parameter gradients
                optimizer.zero_grad();

                # forward + backward + optimize
                try:
                    outputs = torch.softmax(input=net(x),dim=1); #need to check float NaN value?
                    running_acc += (((outputs.data.argmax(dim=1) == y.argmax(dim=1))*1).float().mean()).item();
                    loss = lossFunc(outputs.log(), y);
                    loss.backward();
                    optimizer.step();
                    running_loss += loss.item();
                except ValueError:
                    print(f"error label:train")# {y}
                    print(f"error data:train") # {x}
                    continue

            tr_acc = (running_acc / n_batches)*100;
            tr_loss = running_loss / n_batches;

            #Epoch wise validation Validation
            epoch_train_time = time.time() - epoch_start_time;

            net.eval();
            val_acc, val_loss = self.__validate(net, lossFunc);
            #Save best model
            self.__chk_bestAcc(val_acc, epochIdx, net);
            self.__on_epoch_end(epoch_start_time, epoch_train_time, epochIdx, cur_lr, tr_loss, tr_acc, val_loss, val_acc);

            running_loss = 0;
            running_acc = 0;
            net.train();

        total_time_taken = time.time() - train_start_time;
        print("Execution finished in: {}".format(U.to_hms(total_time_taken)));


    def __chk_bestAcc(self, acc, epochIdx, net):
        print("__chk_bestAcc is called")
        print(f"current best Acc is {self.bestAcc}")
        print(f"pass in acc is {acc}")
        if acc > self.bestAcc:
            self.bestAcc = acc;
            self.bestAccEpoch = epochIdx +1;
            print(f"model saved....., acc: {acc}")
            
    def __on_epoch_end(self, start_time, train_time, epochIdx, lr, tr_loss, tr_acc, val_loss, val_acc):
        epoch_time = time.time() - start_time;
        val_time = epoch_time - train_time;
        line = 'SP-{} Epoch: {}/{} | Time: {} (Train {}  Val {}) | Train: LR {}  Loss {:.2f}  Acc {:.2f}% | Val: Loss {:.2f}  Acc(top1) {:.2f}% | HA {:.2f}@{}\n'.format(
            self.opt.splits, epochIdx+1, self.opt.nEpochs, U.to_hms(epoch_time), U.to_hms(train_time), U.to_hms(val_time),
            lr, tr_loss, tr_acc, val_loss, val_acc, self.bestAcc, self.bestAccEpoch);
        # print(line)
        sys.stdout.write(line);
        sys.stdout.flush();
        
    
    def __get_lr(self, epoch):
        divide_epoch = np.array([self.opt.nEpochs * i for i in self.opt.schedule]);
        decay = sum(epoch > divide_epoch);
        if epoch <= self.opt.warmup:
            decay = 1;
        return self.opt.LR * np.power(0.1, decay);

    def __get_batch(self, index):
        x = self.trainX[index*self.opt.batchSize : (index+1)*self.opt.batchSize];
        y = self.trainY[index*self.opt.batchSize : (index+1)*self.opt.batchSize];
        return x.to(self.opt.device), y.to(self.opt.device);
    
    
    # def __calibrate(self, net):
    #     self.load_train_data();
    #     net.eval();
    #     with torch.no_grad():
    #         for i in range(1,2):
    #             x_pred = None;
    #             for idx in range(math.ceil(len(self.trainX)/self.opt.batchSize)):
    #                 x = self.trainX[idx*self.opt.batchSize : (idx+1)*self.opt.batchSize];
    #                 scores = net(x);
    #                 x_pred = scores.data if x_pred is None else torch.cat((x_pred, scores.data));
    #             x_pred = x_pred.argmax(dim=1);
    #             x_target = self.trainY.argmax(dim=1);
    #             acc = (((x_pred==x_target)*1).float().mean()*100).item();
    #             print('calibrate accuracy is: {:.2f}'.format(acc));
    #     return acc;

    def QuantizeModel(self):
        net = self.__load_model(True);
        # net = self.__load_model(False);
        config = net.ch_config;
        net.eval();
        
        #Fuse modules to
        torch.quantization.fuse_modules(net.sfeb, ['0','1','2'], inplace=True);
        torch.quantization.fuse_modules(net.sfeb, ['3','4','5'], inplace=True);
        torch.quantization.fuse_modules(net.tfeb, ['0','1','2'], inplace=True);
        torch.quantization.fuse_modules(net.tfeb, ['4','5','6'], inplace=True);
        torch.quantization.fuse_modules(net.tfeb, ['7','8','9'], inplace=True);
        torch.quantization.fuse_modules(net.tfeb, ['11','12','13'], inplace=True);
        torch.quantization.fuse_modules(net.tfeb, ['14','15','16'], inplace=True);
        torch.quantization.fuse_modules(net.tfeb, ['18','19','20'], inplace=True);
        torch.quantization.fuse_modules(net.tfeb, ['21','22','23'], inplace=True);
        # torch.quantization.fuse_modules(net.tfeb, ['25','26','27'], inplace=True);
        # torch.quantization.fuse_modules(net.tfeb, ['28','29','30'], inplace=True);
        # torch.quantization.fuse_modules(net.tfeb, ['33','34','35'], inplace=True);

        net.train();
        net.qconfig = torch.quantization.get_default_qconfig('qnnpack')
        torch.backends.quantized.engine = 'qnnpack';
        print(f"net.qconfig : {net.qconfig}");
        torch.quantization.prepare_qat(net, inplace=True);
        
        # Calibrate with the training data
        # self.__calibrate(net);
        self.__train(net);

        #place trained model to cpu
        net.to('cpu');
        # Convert to quantized model
        torch.quantization.convert(net, inplace=True);
        print('Post Training Quantization: Convert done');

        print("Size of model after quantization");
        torch.save(net.state_dict(), "temp.p")
        print('Size (MB):', os.path.getsize("temp.p")/1e6)
        os.remove('temp.p')

        self.load_test_data();
        val_acc = self.__validate_test(net, True, self.testX, self.testY);
        print('Testing: Acc(top1) {:.2f}%'.format(val_acc));
        net.to('cpu');
        # torch.jit.save(torch.jit.script(net), '{}/th/quantized_models/{}.pt'.format(os.getcwd(), self.opt.model_name.format()));
        torch.jit.save(torch.jit.script(net), f"{save_pt_model_path}/{self.opt.model_name}.pt".format());
        torch.save({'weight':net.state_dict(), 'config':net.ch_config}, f"{save_pt_model_path}/uncompressed_qat_models/uncompress_{self.opt.model_name}.pt");
        
        # **************convert to tflite**********  save_tflite_model_s5
        with torch.no_grad():
            # dummy_input = torch.randn(1, 1, 30225, 1); wrong: RuntimeError: quantized::conv2d (qnnpack): each dimension of output tensor should be greater than 0.
            # dummy_input = torch.FloatTensor(quantize_int8(torch.randn(1, 1, 1, inp_len).numpy(),3)); #correct,workable
            dummy_input = torch.FloatTensor(quantize_int8(torch.randn(1, 1, 1, inp_len).numpy()))
            # dummy_input = torch.randn(1, 1, 1, inp_len).numpy()
            # min_val = dummy_input.min()
            # max_val = dummy_input.max()
            # quantized_input = quantize_float32_2_int8(dummy_input, min_val, max_val)
            print("check!!!", type(dummy_input))
            converter = TFLiteConverter(net,
                                        dummy_input,
                                        # torch.tensor(quantized_input, dtype=torch.int8).float(),
                                        quantize_input_output_type='int8',#設定此欄，輸入會強制為int8
                                        fuse_quant_dequant=True,
                                        quantize_target_type='int8',
                                        hybrid_conv=False,
                                        float16_quantization=True,
                                        optimize=5,
                                        tflite_path=f"{save_tflite_model_path}/{self.opt.model_name}.tflite")
            converter.convert()
            print("====================saving??====================")
        
    def TestModel(self, quant=False):
        if quant:
            print(f"the model name:{self.opt.model_name}");
            net = torch.jit.load(f"{save_pt_model_path}/{self.opt.model_name}.pt")
        else:
            print("has not quanted, load unquanted model...");
            net = self.__load_model();
            # calc.summary(net, (1,1,self.opt.inputLength));
        self.load_test_data();
        net.eval();
        val_acc = self.__validate_test(net, False, self.testX, self.testY);
        print('Testing: Acc(top1) {:.2f}%'.format(val_acc));

    def GetModelSize(self):
        orig_net_path = self.opt.model_path;
        print('Full precision model size (KB):', os.path.getsize(orig_net_path)/(1024));
        save_onnx_name = f"{save_pt_model_path}/{self.opt.model_name}.onnx";
        quant_net_path = f"{save_pt_model_path}/has_qat_models/onnx_models/"+save_onnx_name;
        print('Quantized model size (KB):', os.path.getsize(quant_net_path)/(1024))

In [48]:
use_model_in_step4

'/home/sail/sound_project/sound_ai_v2/STEP/step_4/save_pt_model_s4/20240901_04/retrain_4C_after_2nd_pruning_prunratio0.9_2024090104/retrained_4Cmodel_ratio90.0_vaacc88.0_tracc_80.6640625@1287epoch_20240901071124.pt'

In [49]:
def main():
    opt = getOpts();#opts.parse();
    opt.device = 'cpu';
    opt.saveInfo = "uec_v2_sailtest"
    opt.model_path = use_model_in_step4
    timeStr = genDataTimeStr();
    opt.model_name = "qat_model_{}_{}".format(opt.saveInfo,timeStr);
   
    opt.split = 1;
    opt.hasQuated = False;
    display_info(opt);
    trainer = QATTrainer(opt);

    print('Testing performance of the provided model.....');
    trainer.TestModel();

    print('Quantization process is started.....');
    trainer.QuantizeModel();
    print('Quantization done');

    print('Testing quantized model.');
    trainer.TestModel(True);
    print('Finished');

In [50]:
main()

+------------------------------+
| TLACDNet Sound classification
+------------------------------+
| dataset  : uec_iot
| nEpochs  : 3000
| LRInit   : 0.05
| schedule : [0.1, 0.3, 0.6, 0.9]
| warmup   : 10
| batchSize: 32
| nFolds: 1
| Splits: [1]
| Device: cpu
| Model Path: /home/sail/sound_project/sound_ai_v2/STEP/step_4/save_pt_model_s4/20240901_04/retrain_4C_after_2nd_pruning_prunratio0.9_2024090104/retrained_4Cmodel_ratio90.0_vaacc88.0_tracc_80.6640625@1287epoch_20240901071124.pt
| Model Name: qat_model_uec_v2_sailtest_20240902015957
+------------------------------+
length of samples:1018
Testing performance of the provided model.....
has not quanted, load unquanted model...
[8, 40, 23, 51, 57, 81, 90, 89, 7]
check [8, 40, 23, 51, 57, 81, 90, 89, 7]
+----------------------------------------------------------------------------+
+                           Pytorch Model Summary                            +
------------------------------------------------------------------------------

/tmp/ipykernel_214951/3353999771.py:136: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state = torch.load(self.opt.model_path, map_location=self.opt.device);


Calibration dataset is ready
shape of y_pred: torch.Size([100, 7])
shape of y_target: torch.Size([100, 7])
__chk_bestAcc is called
current best Acc is 0.0
pass in acc is 29.0
model saved....., acc: 29.0
SP-[1] Epoch: 1/3000 | Time: 0m06s (Train 0m06s  Val 0m00s) | Train: LR 0.005000000000000001  Loss 1.35  Acc 23.73% | Val: Loss 1.80  Acc(top1) 29.00% | HA 29.00@1
shape of y_pred: torch.Size([100, 7])
shape of y_target: torch.Size([100, 7])
__chk_bestAcc is called
current best Acc is 29.0
pass in acc is 96.0
model saved....., acc: 96.0
SP-[1] Epoch: 2/3000 | Time: 0m06s (Train 0m06s  Val 0m00s) | Train: LR 0.005000000000000001  Loss 1.24  Acc 30.96% | Val: Loss 0.37  Acc(top1) 96.00% | HA 96.00@2
shape of y_pred: torch.Size([100, 7])
shape of y_target: torch.Size([100, 7])
__chk_bestAcc is called
current best Acc is 96.0
pass in acc is 87.0
SP-[1] Epoch: 3/3000 | Time: 0m06s (Train 0m06s  Val 0m00s) | Train: LR 0.005000000000000001  Loss 0.98  Acc 48.93% | Val: Loss 0.50  Acc(top1) 87.

INFO (tinynn.converter.base) Generated model saved to /home/sail/sound_project/sound_ai_v2/STEP/step_5/save_tflite_model_s5/20240902_01/qat_model_uec_v2_sailtest_20240902015957.tflite


====================saving??====================
Quantization done
Testing quantized model.
the model name:qat_model_uec_v2_sailtest_20240902015957
device is :cpu
len of Y:100
torch.Size([100, 7])
Testing: Acc(top1) 16.00%
Finished
